In [19]:

import sys
import numpy as np

import pickle as pkl
import torch
from os.path import join as oj
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import torch.utils.data as utils
import sys, time
from torch.utils.data import TensorDataset, ConcatDataset
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('precision', 3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
save_path = "../results_for_export"
trained_model_folder = '../models/cifar10_equal_loss'
fnames = [oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder)]

results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects() 

In [21]:
results['final_acc'] = [x[-1] for x in results['accs_test']] 
results['final_acc_train'] = [x[-1] for x in results['accs_train']]
results['final_cd'] = [x[-1] for x in results['cd']]
results['final_test_loss'] = [x[-1] for x in results['losses_test']]
results['final_train_loss'] = [min(x) for x in results['losses_train']]


In [22]:
results.reset_index(drop=True, inplace=True)
results_save = results[['regularizer_rate','final_acc','final_acc_train','final_test_loss', 'final_cd', 'method','seed' ]].sort_values(by = ['regularizer_rate'])

In [37]:

from model import Net
import pickle
with open('data_stylized2.pkl', 'rb') as fo:
    data_dict = pickle.load(fo, encoding='bytes')

In [38]:
x_style =  torch.tensor((data_dict['cifar_stylized']-0.5)/0.5).float()
x= torch.tensor((data_dict['cifar']-0.5)/0.5).float()
y= torch.tensor(np.asarray(data_dict['labels']))

In [40]:
stylized_set  = TensorDataset(x_style,y)
stylized_loader = torch.utils.data.DataLoader(stylized_set, batch_size=256, 
                                             shuffle=False, num_workers=2)
test_set = TensorDataset(x,y)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, 
                                             shuffle=False, num_workers=2)

In [41]:
def test( model,  data_loader, device = 'cuda'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    return 100. * correct / len(data_loader.dataset)

In [42]:

my_net= Net()
my_net = my_net.eval().cuda()

In [43]:
style_list = []
test_list =[]
for i in range(len(results)):

    my_net.load_state_dict(results.iloc[i]['model_weights'])
    style_list.append(test(my_net, stylized_loader))
    test_list.append(test(my_net, test_loader))
results_save['style_acc'] = [x for x in style_list]
results_save['test_acc'] = [x for x in test_list]

In [44]:
concise_results = results_save.groupby(by = [ 'method','regularizer_rate']).mean()
concise_results[[ 'style_acc', 'test_acc','final_acc','final_acc_train','final_test_loss', ]]

style_acc  test_acc  final_acc  final_acc_train  \
method regularizer_rate                                                    
CDEP   0.000e+00            53.333    76.667     76.607           78.516   
       1.000e-28            56.000    76.000     72.840           87.500   
       1.000e+00            59.000    77.333     72.840           87.500   
       1.000e+01            55.000    79.000     72.840           87.500   

                         final_test_loss  
method regularizer_rate                   
CDEP   0.000e+00                   0.684  
       1.000e-28                   0.814  
       1.000e+00                   0.814  
       1.000e+01                   0.814